# Landsat 8 Classification Example

This example will walk you through an example of using DELTA to train a simple example model. You can use what you learn here to use DELTA on your own datasets and with your own model architectures.
In this example you will:
- Download a dataset of images and labels
- Train a simple model using example configuration files
- Examine results of the trained model
- Make some changes to the configuration files and train a modified model
- Examine the results of the modified model

## Downloading and Extracting Dataset

The dataset includes satellite images along with classification labels for different types of land cover (water, cloud, snow, etc.).


In [1]:
!echo "Downloading dataset."
!curl -O https://landsat.usgs.gov/cloud-validation/sparcs/l8cloudmasks.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1483M  100 1483M    0     0  4333k      0  0:05:50  0:05:50 --:--:-- 6467k  0  0:03:21  0:00:34  0:02:47 6195kk 0  6080k      0  0:04:09  0:00:44  0:03:25 1240k164k      0  0:06:04  0:01:12  0:04:52 1861k2M    0     0  4104k      0  0:06:10  0:01:20  0:04:50 3942k 0  0:05:48  0:01:33  0:04:15 6833k46  0:03:54 5871k      0  0:05:22  0:01:56  0:03:26 7779k0  0:05:16  0:02:50  0:02:26 4334k      0  0:05:38  0:03:44  0:01:54  870k0  0:05:42  0:03:47  0:01:55  586k  0:03:48  0:01:55  521k0  0:00:40 7809k05:39  0:00:16 4483k


Here we're extracting the dataset and organizing the images into folders.
We are:
- setting aside two images in a folder called "validate" to test our model later
- moving the satellite images into a folder called "train"
- moving the classification labels into a folder called "labels"

In [2]:
!echo "Extracting dataset."
!unzip -q l8cloudmasks.zip
!mkdir validate
!mv sending/LC82290562014157LGN00_24_data.tif sending/LC82210662014229LGN00_18_data.tif validate/
!mkdir train
!mv sending/*_data.tif train/
!mkdir labels
!mv sending/*_mask.png labels/

Extracting dataset.
mkdir: labels: File exists


In [3]:
!ls

example_screenshots            l8_cloud_train_parameters.yaml
l8_cloud.sh                    l8cloudmasks.zip
l8_cloud_dataset.yaml          labels
l8_cloud_dataset_water.yaml    sending
l8_cloud_example.ipynb         train
l8_cloud_train_network.yaml    validate


In [4]:
!ls train/ | head

LC80010812013365LGN00_18_data.tif
LC80020622013244LGN00_32_data.tif
LC80050152014172LGN00_12_data.tif
LC80050562014076LGN00_33_data.tif
LC80150242014146LGN00_23_data.tif
LC80190352014078LGN01_26_data.tif
LC80200462014213LGN00_11_data.tif
LC80250402013245LGN00_45_data.tif
LC80250482014072LGN00_18_data.tif
LC80310432013207LGN00_11_data.tif


### Example Satellite Image
<img src='example_screenshots/Screen Shot 2021-07-13 at 6.57.11 PM.png'>

In [5]:
!ls labels | head

LC80010812013365LGN00_18_mask.png
LC80020622013244LGN00_32_mask.png
LC80050152014172LGN00_12_mask.png
LC80050562014076LGN00_33_mask.png
LC80150242014146LGN00_23_mask.png
LC80190352014078LGN01_26_mask.png
LC80200462014213LGN00_11_mask.png
LC80250402013245LGN00_45_mask.png
LC80250482014072LGN00_18_mask.png
LC80310432013207LGN00_11_mask.png


### Example Label Image

The different colors represent different land cover classifications.

<img src='labels/LC80010812013365LGN00_18_mask.png'>

## Training a Model

We'll use the following configuration files which specify a dataset to use (l8_cloud_dataset.yaml), a network architecture to train (l8_cloud_train_network.yaml), and the parameters to train with (l8_cloud_train_parameters.yaml).

### Configuration YAMLs

<a href='./l8_cloud_dataset.yaml' > l8_cloud_dataset.yaml </a><br />
<a href='./l8_cloud_train_network.yaml' > l8_cloud_train_network.yaml </a><br />
<a href='./l8_cloud_train_parameters.yaml' > l8_cloud_train_parameters.yaml </a>

<a href='../../../delta/config/README.md' > Detailed Config Documentation </a>

In [8]:
!delta train --config l8_cloud_dataset.yaml --config l8_cloud_train_network.yaml --config l8_cloud_train_parameters.yaml l8_clouds.SavedModel 2>/dev/null

2021-11-10 11:09:58.657216: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-10 11:09:58.686678: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fcdb2e457a0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-11-10 11:09:58.686712: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
Epoch 1/10
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
31/31 [==============================] - 372s 12s/step - loss: 0.9725 - sparse_categorical_accuracy: 0.6823
Epoch 2/10
31/31 [==============================] - ETA: 0s - loss: 0.6424 - sparse_categorical_accuracy: 0.7887WARNING:tensorflow:From 

## Examine Model Results

The previous step produced a trained model. Now we can use the model to classify the images we set aside in the "validate" folder.

In [26]:
!delta classify --config l8_cloud_dataset.yaml --image-dir ./validate --outdir ./model_output --overlap 32 l8_clouds.SavedModel 2>/dev/null

LC82290562014157LGN00_24_data : |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX| 4 / 4
Shadow              --- Precision:  71.66%    Recall:  18.84%        Pixels: 140931 / 1048576
Shadow over Water   --- Precision:   0.00%    Recall:   0.00%        Pixels: 41893 / 1048576
Water               --- Precision:  84.42%    Recall:  94.72%        Pixels: 259184 / 1048576
Snow                --- Precision:   0.00%    Recall:   0.00%        Pixels: 0 / 1048576
Land                --- Precision:  57.91%    Recall:  99.38%        Pixels: 307181 / 1048576
Cloud               --- Precision:  98.77%    Recall:  63.86%        Pixels: 299387 / 1048576
Flooded             --- Precision:   0.00%    Recall:   0.00%        Pixels: 0 / 1048576
 73.29% Correct
LC82210662014229LGN00_18_data : |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX| 4 / 4
Shadow              --- Precision:  99.59%    Recall:   8.21%        Pixels: 107635 / 1048576
S

### Validation Image
The validation image we fed into the model.

<img src='example_screenshots/Screen Shot 2021-07-14 at 1.35.36 PM.png'>

### Validation Model Output
The output from the model classifying the land cover in the above image.

<img src='example_screenshots/Screen Shot 2021-07-14 at 1.36.41 PM.png'>

## Train Modified Model

The previous model we trained classified all the land cover types in the landsat 8 dataset (water, snow, clouds, etc.). Now we're going to make a simple modification to the dataset configuration file and train a model that just classifies water in the satellite image.

All we have to do is add one of DELTA's built in preprocessing functions. It will map all the of the classes except water to one class and set water as the other class.

<a href='./l8_cloud_dataset_water.yaml' > l8_cloud_dataset_water.yaml </a><br />

Excerpt from l8_cloud_dataset_water.yaml :
```yaml
# ______________
# this mapping section tells DELTA to set all
# the classes EXCEPT water to 0 and the water
# claass to 2
# ______________
    preprocess:
      - substitute:
          mapping:
            - 0
            - 0
            - 2
            - 0
            - 0
            - 0
            - 0
  classes:
    - 0:
        name: Shadow
        color: 0x000000
    - 1:
        name: Shadow over Water
        color: 0x000080
    - 2:
        name: Water
        color: 0x0000FF
    - 3:
        name: Snow
        color: 0x00FFFF
    - 4:
        name: Land
        color: 0x808080
    - 5:
        name: Cloud
        color: 0xFFFFFF
    - 6:
        name: Flooded
        color: 0x808000
```

In [39]:
!delta train --config l8_cloud_dataset_water.yaml --config l8_cloud_train_network.yaml --config l8_cloud_train_parameters.yaml l8_clouds_water.SavedModel 2>/dev/null

Epoch 1/5
31/31 [==============================] - 31s 505ms/step - loss: 0.5149 - sparse_categorical_accuracy: 0.8845
Epoch 2/5
31/31 [==============================] - 16s 459ms/step - loss: 0.1606 - sparse_categorical_accuracy: 0.9453
Epoch 3/5
31/31 [==============================] - 16s 450ms/step - loss: 0.1455 - sparse_categorical_accuracy: 0.9429
Epoch 4/5
31/31 [==============================] - 16s 441ms/step - loss: 0.1311 - sparse_categorical_accuracy: 0.9501
Epoch 5/5
31/31 [==============================] - 17s 458ms/step - loss: 0.1471 - sparse_categorical_accuracy: 0.9440

Finished, saving model to file:///home/mvonpohl/.local/share/delta/mlflow/1/66114b61efaf4bde871df044bf9b1dbc/artifacts/final_model.savedmodel.
Elapsed time =  105.85115361213684


## Examine Model Results

Now we can examine the new model by classifying the images we set aside in the "validate" folder.

In [43]:
!delta classify --config l8_cloud_dataset_water.yaml --image-dir ./validate --outdir ./model_output_water --overlap 32 l8_clouds_water.SavedModel 2>/dev/null

LC82290562014157LGN00_24_data : |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX| 9 / 9
Shadow              --- Precision:  98.98%    Recall:  95.22%        Pixels: 1830837 / 2167040
Shadow over Water   --- Precision:   0.00%    Recall:   0.00%        Pixels: 0 / 2167040
Water               --- Precision:  78.44%    Recall:  94.68%        Pixels: 336203 / 2167040
Snow                --- Precision:   0.00%    Recall:   0.00%        Pixels: 0 / 2167040
Land                --- Precision:   0.00%    Recall:   0.00%        Pixels: 0 / 2167040
Cloud               --- Precision:   0.00%    Recall:   0.00%        Pixels: 0 / 2167040
Flooded             --- Precision:   0.00%    Recall:   0.00%        Pixels: 0 / 2167040
 95.14% Correct
LC82210662014229LGN00_18_data : |XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX| 9 / 9
Shadow              --- Precision: 100.00%    Recall: 100.00%        Pixels: 2167040 / 2167040
Shadow over W

### Validation Image
The validation image we fed into the model.

<img src='example_screenshots/Screen Shot 2021-07-14 at 1.35.36 PM.png'>

### Validation Model Output
The output from the model classifying water coverage in the above image.

<img src='example_screenshots/Screen Shot 2021-07-14 at 2.17.03 PM.png'>

